In [2]:
%pip install pandas numpy matplotlib scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [4]:
df = pd.read_csv('dropped_data.csv')

In [5]:
label_encoder = LabelEncoder()

# eliminate unnecessary floats
df['year'] = df['year'].astype(int)
df['odometer'] = df['odometer'].astype(int)

# label encoding for columns with many unique values
label_encoder_condition = LabelEncoder()
label_encoder_size = LabelEncoder()
label_encoder_manufacturer = LabelEncoder()
label_encoder_model = LabelEncoder()
label_encoder_paint_color = LabelEncoder()
label_encoder_type = LabelEncoder()

df['condition'] = label_encoder_condition.fit_transform(df['condition'])
df['size'] = label_encoder_size.fit_transform(df['size'])
df['manufacturer'] = label_encoder_manufacturer.fit_transform(df['manufacturer'])
df['model'] = label_encoder_model.fit_transform(df['model'])
df['paint_color'] = label_encoder_paint_color.fit_transform(df['paint_color'])
df['type'] = label_encoder_type.fit_transform(df['type'])

# one hot encoding for columns with few unique values
df = pd.get_dummies(df, columns=['fuel', 'transmission', 'title_status', 'drive'], drop_first=True)

# drop unnecessary columns, clean cylinders column
df.drop(columns=["id", "posting_date"], inplace=True)
df['cylinders'] = df['cylinders'].str.replace('cylinders', '').str.strip()
most_common_cylinders = df.loc[df['cylinders'] != 'other', 'cylinders'].mode()[0]
df['cylinders'] = df['cylinders'].replace('other', most_common_cylinders)

df.head()

,price,year,manufacturer,model,condition,cylinders,odometer,size,type,paint_color,...,fuel_other,transmission_manual,transmission_other,title_status_lien,title_status_missing,title_status_parts only,title_status_rebuilt,title_status_salvage,drive_fwd,drive_rwd
0,15000,2013,13,3869,0,6,128000,1,10,0,...,False,False,False,False,False,False,False,False,False,True
1,19900,2004,13,4233,2,8,88000,1,8,1,...,False,False,False,False,False,False,False,False,False,False
2,14000,2012,16,6337,0,6,95000,1,5,9,...,False,False,False,False,False,False,False,False,True,False
3,22500,2001,13,4365,2,8,144700,1,10,10,...,False,True,False,False,False,False,False,False,False,True
4,15000,2017,10,2242,0,8,90000,2,9,5,...,False,False,False,False,False,False,True,False,False,True


In [6]:
max_price_row = df.loc[df['price'].idxmax()]

encoded_manufacturer = max_price_row['manufacturer']
encoded_model = max_price_row['model']

actual_manufacturer = label_encoder_manufacturer.inverse_transform([encoded_manufacturer])[0]
actual_model = label_encoder_model.inverse_transform([encoded_model])[0]

print(f"Manufacturer: {actual_manufacturer}, Model: {actual_model}, Price: {max_price_row['price']}")

Manufacturer: toyota, Model: tundra, Price: 3736928711


In [7]:
# comparing statistical values with and without top 1% to look for outliers

print("With outlier:")
print(df['price'].describe())

threshold = df['price'].quantile(0.99)  # threshold for top 1% of cars in terms of price
df_without_outlier = df[df['price'] < threshold]

print("\nWithout outlier:")
print(df_without_outlier['price'].describe())


With outlier:
count    7.919500e+04
mean     7.943274e+04
std      1.387575e+07
min      0.000000e+00
25%      4.950000e+03
50%      9.000000e+03
75%      1.750000e+04
max      3.736929e+09
Name: price, dtype: float64

Without outlier:
count    78398.000000
mean     12525.120335
std      10899.472885
min          0.000000
25%       4900.000000
50%       8995.000000
75%      16995.000000
max      55999.000000
Name: price, dtype: float64


In [8]:


top_1_percent_cars = df[df['price'] > threshold].copy()

# reverse label encoding to view original values for top 1% priced cars
top_1_percent_cars['manufacturer'] = label_encoder_manufacturer.inverse_transform(top_1_percent_cars['manufacturer'])
top_1_percent_cars['model'] = label_encoder_model.inverse_transform(top_1_percent_cars['model'])
top_1_percent_cars['condition'] = label_encoder_condition.inverse_transform(top_1_percent_cars['condition'])
top_1_percent_cars['size'] = label_encoder_size.inverse_transform(top_1_percent_cars['size'])
top_1_percent_cars['paint_color'] = label_encoder_paint_color.inverse_transform(top_1_percent_cars['paint_color'])
top_1_percent_cars['type'] = label_encoder_type.inverse_transform(top_1_percent_cars['type'])

# sort df by price in descending order (highest to lowest)
top_1_percent_cars_sorted = top_1_percent_cars.sort_values(by='price', ascending=False)

# view all rows and cols
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 

print(top_1_percent_cars_sorted)


            price  year   manufacturer                             model  \
60291  3736928711  2007         toyota                            tundra   
5128   1111111111  1999           ford            f350 super duty lariat   
42568   135008900  2008         nissan                  titan se kingcab   
58487   123456789  1996            gmc                       sierra 2500   
24685   123456789  1999          buick                             regal   
24477    17000000  2007            ram                              2500   
11930     2000000  2002         saturn           l-series l200 4dr sedan   
27826     1234567  2006           jeep                          wrangler   
40645     1111111  1970          dodge                        challenger   
37235      195000  2017         subaru                               wrx   
18661      169999  2010        ferrari                        458 italia   
20292      165000  2018          dodge                         srt demon   
61093      1

In [9]:
# sort df and remove top 9 values (obviously outliers/fillers)
df = df.sort_values(by='price', ascending=False)
df = df.iloc[9:].copy()  
print(df.head(10))  

        price  year  manufacturer  model  condition cylinders  odometer  size  \
37235  195000  2017            36   9187          3         4     73000     1   
18661  169999  2010            11    724          0         8     13000     1   
20292  165000  2018            10   8099          4         8        12     1   
61093  155000  2020            26   1782          4         6       250     1   
68588  150000  1959            19   9378          0         6     64765     0   
13182  150000  1959            19   9378          0         6     64765     0   
10727  150000  1959            19   9374          0         6     64765     0   
15622  150000  2009            13   3536          2         4    182415     1   
7120   150000  1959            19   9373          0         6     64765     0   
69245  144832  2013            40   7287          2         5    144832     2   

       type  paint_color  fuel_electric  fuel_gas  fuel_hybrid  fuel_other  \
37235     9            5      

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

y = df['price']
X = df.drop(columns=['price'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()

model.fit(X_train, y_train)

LinearRegression()

In [11]:
from sklearn.metrics import mean_absolute_error, r2_score

y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae}")

r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")

MAE: 7136.304877580258
R-squared: 0.28469741411462746


In [12]:
# trying more advanced model to get better MAE and r^2 results
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [13]:
y_pred_rf = rf_model.predict(X_test)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
print(f"RF - mae: {mae_rf}")

r2_rf = r2_score(y_test, y_pred_rf)
print(f"RF - Rsquared: {r2_rf}")

RF - mae: 2681.001227713523
RF - Rsquared: 0.7931235631444478


In [14]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    'n_estimators': [100, 200, 500, 1000],         
    'max_depth': [None, 10, 20, 30, 40],           
    'min_samples_split': [2, 5, 10],               
    'min_samples_leaf': [1, 2, 4],                 
    'bootstrap': [True, False]                     
}

random_search = RandomizedSearchCV(estimator=rf_model, param_distributions=param_grid, n_iter=10, cv=3, verbose=2, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)
print(f'Best hyperparameters: {random_search.best_params_}')

best_rf_model = random_search.best_estimator_
y_pred_best_rf = best_rf_model.predict(X_test)
mae_best_rf = mean_absolute_error(y_test, y_pred_best_rf)
r2_best_rf = r2_score(y_test, y_pred_best_rf)
print(f'Best Random Forest - Mean Absolute Error: {mae_best_rf}')
print(f'Best Random Forest - R-squared: {r2_best_rf}')

Fitting 3 folds for each of 10 candidates, totalling 30 fits


c:\Python312\Lib\site-packages\numpy\ma\core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best hyperparameters: {'n_estimators': 1000, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': None, 'bootstrap': True}
Best Random Forest - Mean Absolute Error: 2772.539582685514
Best Random Forest - R-squared: 0.7894164441139357


In [15]:
from sklearn.ensemble import GradientBoostingRegressor

gb_model = GradientBoostingRegressor(random_state=42)
gb_model.fit(X_train, y_train)

GradientBoostingRegressor(random_state=42)

In [16]:
y_pred_gb = gb_model.predict(X_test)

mae_gb = mean_absolute_error(y_test, y_pred_gb)
print(f'GB - MAE: {mae_gb}')

r2_gb = r2_score(y_test, y_pred_gb)
print(f'GB - Rsquared: {r2_gb}')

GB - MAE: 4520.31775355424
GB - Rsquared: 0.6154145922716587


In [17]:
param_grid_gb = {
    'n_estimators': [100, 200, 500],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

random_search_gb = RandomizedSearchCV(estimator=gb_model, param_distributions=param_grid_gb, n_iter=10, cv=3, verbose=2, random_state=42, n_jobs=-1)

random_search_gb.fit(X_train, y_train)

print(f'Best hyperparameters: {random_search_gb.best_params_}')

best_gb_model = random_search_gb.best_estimator_
y_pred_best_gb = best_gb_model.predict(X_test)

mae_best_gb = mean_absolute_error(y_test, y_pred_best_gb)
r2_best_gb = r2_score(y_test, y_pred_best_gb)

print(f'Best Gradient Boosting - Mean Absolute Error: {mae_best_gb}')
print(f'Best Gradient Boosting - R-squared: {r2_best_gb}')

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best hyperparameters: {'n_estimators': 500, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_depth': 7, 'learning_rate': 0.05}
Best Gradient Boosting - Mean Absolute Error: 3345.4516885880294
Best Gradient Boosting - R-squared: 0.7638214179706679


In [18]:
# best model was untuned RF model, so I will use it as the final model

final_rf_model = RandomForestRegressor(random_state=42)
final_rf_model.fit(X, y)

RandomForestRegressor(random_state=42)

In [19]:
def test_random_row(model, df):
    random_row = df.sample(n=1)

    actual_price = random_row['price'].values[0]  
    test_features = random_row.drop(columns=['price'])  
    car_manufacturer = random_row['manufacturer'].values[0]
    car_model = random_row['model'].values[0]

    print("Testing with the following features:")
    print(test_features)
    
    predicted_price = model.predict(test_features)[0]
    
    print(f"\nActual Price: ${actual_price:,.2f}")
    print(f"Predicted Price: ${predicted_price:,.2f}")
    print(f"Manufacturer: {car_manufacturer}")
    print(f"Model: {car_model}")


test_random_row(final_rf_model, df)


Testing with the following features:
       year  manufacturer  model  condition cylinders  odometer  size  type  \
63561  2015            13   3190          0         8     68095     1    10   

       paint_color  fuel_electric  fuel_gas  fuel_hybrid  fuel_other  \
63561           10          False      True        False       False   

       transmission_manual  transmission_other  title_status_lien  \
63561                False               False              False   

       title_status_missing  title_status_parts only  title_status_rebuilt  \
63561                 False                    False                 False   

       title_status_salvage  drive_fwd  drive_rwd  
63561                 False      False       True  

Actual Price: $28,000.00
Predicted Price: $27,322.48
Manufacturer: 13
Model: 3190


In [20]:
unencoded_manufacturer = label_encoder_manufacturer.inverse_transform([17])
unencoded_model = label_encoder_model.inverse_transform([7332])
print(f"Manufacturer: {unencoded_manufacturer}\nModel: {unencoded_model}")


Manufacturer: ['hyundai']
Model: ['santa fe']
